<b><font color="red" size="6">ch14. 웹데이터 수집</font></b>
# 1절. Selenium을 이용한 동적 웹크롤링 문법
- https://selenium-python.readthedocs.io/index.html
<br>`pip install selenium`

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [ ]:
dv = webdriver.Chrome()
dv.get('http://python.org')
elem = dv.find_element(By.NAME, 'q')
elem.clear()
elem.send_keys('pycon')

In [ ]:
from selenium.webdriver.common.keys import Keys
elem.send_keys(Keys.RETURN) # enter키 효과

In [ ]:
btn_elem = dv.find_element(By.CSS_SELECTOR, 'button#submit')
btn_elem.click()

In [ ]:
result_list = dv.find_elements(By.CSS_SELECTOR, 'li > h3 > a')
for result in result_list:
    print("%s - %s" % (result.text, result.get_attribute('href')))

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(dv.page_source, 'html.parser')
result_list = soup.select('li > h3 > a')
for result in result_list:
    print('%s - %s' % (result.text, result.attrs['href']))

In [ ]:
from urllib.parse import urlparse
current_url = dv.current_url # 셀레니윰을 통해 접근한 현재 url
print('현재주소 :',current_url)
parse_url = urlparse(current_url)
print(parse_url,parse_url.scheme, parse_url.netloc)
domain = f'{parse_url.scheme}://{parse_url.netloc}'
domain = '{}://{}'.format(parse_url.scheme, parse_url.netloc)
print('도메인 :',domain)

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(dv.page_source, 'html.parser')
result_list = soup.select('li > h3 > a')
for result in result_list:
    print('%s - %s' % (result.text, domain+result.attrs['href']))

In [ ]:
dv.close() # 브라우저 종료

# 2절. 동적웹크롤링 예제
## 2-1 다음 뉴스 검색

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome()
url = 'https://daum.net'
driver.get(url)
# driver.implicitly_wait(0.5) # 0.5초 요소를 찾을 때까지 대기시간
time.sleep(0.5) # 0.5초 대기
driver.find_element(By.CLASS_NAME, 'tf_keyword').click()
query = input('검색할 단어는 ?')
driver.find_element(By.CSS_SELECTOR, 'input[type="text"]').send_keys(query)
driver.find_element(By.CLASS_NAME, 'btn_ksearch').click()

In [ ]:
# 뉴스 탭 클릭
driver.find_elements(By.CSS_SELECTOR, 'ul.list_tab > li')[1].click()

In [ ]:
news_list = []
bodies = driver.find_elements(By.CSS_SELECTOR, 'div.item-title > strong.tit-g.clamp-g')
bodies = driver.find_elements(By.CSS_SELECTOR, 'div[class="item-title"] > strong.tit-g.clamp-g')
# print(len(bodies))

for body in bodies:
    a = body.find_element(By.TAG_NAME, 'a')
    title = a.text
    link = a.get_attribute('href')
    news_list.append([title, link])
    print(title,link)

In [ ]:
page_nav = driver.find_element(By.CSS_SELECTOR, 'div.inner_paging')
next_page = page_nav.find_element(By.LINK_TEXT, '2')
next_page.click()

## 2-2 페이징 처리
- 다음 뉴스 페이징 처리

In [ ]:
# 원하는 페이지 만큼 검색하여 검색 결과 데이터프레임으로 출력
driver = webdriver.Chrome()
time.sleep(1)

url = 'https://daum.net'
driver.get(url)
# driver.implicitly_wait(0.5) # 0.5초 요소를 찾을 때까지 대기시간
driver.find_element(By.CLASS_NAME, 'tf_keyword').click()
query = input('검색할 단어는 ?')
driver.find_element(By.CSS_SELECTOR, 'input[type="text"]').send_keys(query)
driver.find_element(By.CLASS_NAME, 'btn_ksearch').click()

# 뉴스 탭 클릭
driver.find_elements(By.CSS_SELECTOR, 'ul.list_tab > li')[1].click()

news_list = []

page_numbers = int(input('몇 페이지 크롤링 할까요?'))
for page_number in range(1, page_numbers+1):
    
    bodies = driver.find_elements(By.CSS_SELECTOR, 'div.item-title > strong.tit-g.clamp-g')
    bodies = driver.find_elements(By.CSS_SELECTOR, 'div[class="item-title"] > strong.tit-g.clamp-g')
    # print(len(bodies))
    print(f'----------------{page_number}page 크롤링 결과----------------')
    for body in bodies:
        a = body.find_element(By.TAG_NAME, 'a')
        title = a.text
        link = a.get_attribute('href')
        news_list.append([title, link])
        print(title,link)

    page_nav = driver.find_element(By.CSS_SELECTOR, 'div.inner_paging')
    next_page = page_nav.find_element(By.LINK_TEXT, str(page_number+1))
    next_page.click()
    time.sleep(1)
driver.close()
import pandas as pd
display(pd.DataFrame(news_list, columns=['title','link']).head())


## 2-3 맞춤법 검사기
- 네이버 맞춤법 검사기 이용

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

In [ ]:
driver = webdriver.Chrome()
time.sleep(0.5)
driver.get('https://www.naver.com/')
elem = driver.find_element(By.CSS_SELECTOR, 'input#query')
elem.send_keys('맞춤법 검사기')
elem.send_keys(Keys.RETURN) # enter 키

textarea = driver.find_element(By.CLASS_NAME, 'txt_gray')
textarea.send_keys('안녕하세요. 안뇽하세요. 방갑습니다')

button = driver.find_element(By.CLASS_NAME, 'btn_check')
button.click()

time.sleep(0.5)
soup = BeautifulSoup(driver.page_source, 'html.parser')
result = soup.select_one('p._result_text.stand_txt').text
driver.close()
result

### 맞춤법 검사 전 text파일을 맞춤법 검사 완료된 text파일로 output

In [ ]:
# f = open('data/ch14_맞춤법검사전.txt','r',encoding='utf-8')
# text = f.read()
# f.close()

In [ ]:
with open('data/ch14_맞춤법검사전.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
ready_list = []
while(len(text) > 300):
    temp = text[:300]
    last_dot_index = temp.rfind('\n')
    ready_list.append(text[:last_dot_index+1])
    print(last_dot_index)
    text = text[last_dot_index+1 : ]
ready_list.append(text[1:])

In [ ]:
# ready_list 확인
[len(ready) for ready in ready_list]

In [ ]:
driver = webdriver.Chrome()
time.sleep(1)
driver.get('https://www.naver.com')
elem = driver.find_element(By.NAME,'query')
elem.send_keys('맞춤법 검사기')
elem.send_keys(Keys.RETURN)
time.sleep(1) # 1초 대기
# driver.implicitly_wait(1)

results = '' # 맞춤법 검사 완료된 text
textarea = driver.find_element(By.CLASS_NAME,'txt_gray')

for ready in ready_list:
    textarea.send_keys(Keys.CONTROL, 'a') # ctrl + a
    textarea.send_keys(ready)
    button = driver.find_element(By.CLASS_NAME,'btn_check')
    button.click()
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    result = soup.select_one('p._result_text').text
    results += result+'\n'
driver.close()

In [ ]:
print(results)

In [ ]:
# 맞춤법 결과 result를 data/ch14_맞춤법검사후.txt로 출력
with open('data/ch14_맞춤법검사후.txt','w',encoding='utf-8')as f:
    f.write(results)

# 연습문제

In [30]:
# 필요한 package import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time


# 파일 읽어오기
with open('data/ch14_맞춤법검사후.txt','r',encoding='utf-8')as f:
    text=f.read()


# 검색할 text 재단
text_list=[]
while(len(text)>1000):
    temp = text[:1000]
    last_dot_index=temp.rfind('.')
    text_list.append(temp[:last_dot_index+1])
    text=text[last_dot_index+1:]
text_list.append(text)


# 브라우저 실행
driver = webdriver.Chrome()
driver.get('https://translate.kakao.com/')


# 검색과 for문 결합
results = ''
for text in text_list:
    # 검색할 text 입력 및 버튼 실행
    elem = driver.find_element(By.CLASS_NAME, 'translate_area')
    elem.clear()
    elem.send_keys(text)
    button = driver.find_element(By.CLASS_NAME, 'btn_translate')
    button.click()
    # 검색후 결과 읽어오기
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    results += soup.select_one('div.result_area').text+'\n'
    
#브라우저 종료 후 파일 출력
driver.close()
with open('data/ch14_영문번역.txt','w',encoding='utf-8') as f:
    f.write(results)

# 3절. 자동화
## 3-1. 컴퓨터가 파이썬을 실행
    - unix : corn
    - win : 작업스케쥴러
        * window + R : taskschd.msc
## 3-2. 파이썬 프로그램에서 스케쥴링
    - pip install schedule
        

In [31]:
import schedule
import time

def job():
    print('작업이 실행되었습니다.')
# job함수는 10초마다 수행할 거야
schedule.every(10).seconds.do(job)

# schedule.every().day.at('17:59').do(job)
# schedule.every().hour.do(job)
# schedule.every(1).minutes.do(job)

# 무한 루프를 돌며 스케쥴된 작업이 있는지 확인하고 자동화
while True:
    schedule.run_pending()
    time.sleep(1)

작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.
작업이 실행되었습니다.


KeyboardInterrupt: 